In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas.tools.plotting import lag_plot
from pandas.tools.plotting import autocorrelation_plot
from fbprophet import Prophet

plt.style.use('fivethirtyeight')

In [ ]:
print('Reading data...')

key_1 = pd.read_csv('../input/key_1.csv')
print('Key file read')
train_1 = pd.read_csv('../input/train_1.csv')
print('Training data read')
ss_1 = pd.read_csv('../input/sample_submission_1.csv')

print ('All data has been read')

Having read in the data, I'll drop the first column, which holds the Page names, so that I can more easily manipulate the columns, and convert the strings into Pandas datetime objects.

In [ ]:
df = train_1.drop(train_1.columns[0], axis=1)
df.columns = pd.to_datetime(df.columns)
# list(df)

Now that we have indexing by DateTime objects, we can use a resampling technique to group all the data by month rather than by day, and plot that individual Series (change the `iloc` to whatever you want).

In [ ]:
series = df.iloc[1]
series = series.resample('MS').mean()
series.plot()
plt.show()
autocorrelation_plot(series)
plt.show()
lag_plot(series)

After checking out a few row plots, one thing jumps out at me: there's a huge seasonality to some of these page visits. 

Facebook has a library called Prophet. Below I'm pulling out one row, turning it into a DataFrame, and renaming the columns the way that Prophet expects.

In [ ]:
df = train_1.drop(train_1.columns[0], axis=1)
df.columns = pd.to_datetime(df.columns)
series = df.iloc[1]
df = series.to_frame().reset_index()
df = df.rename(columns={'index': 'ds', 1: 'y'})
df.head()

In [ ]:
m = Prophet()
m.fit(df);
future = m.make_future_dataframe(periods=30)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
m.plot(forecast)
m.plot_components(forecast);